### Libraries

In [ ]:
pip install boto3

In [ ]:
pip install openai

In [ ]:
import boto3
import pandas as pd
import os
import numpy as np
import nltk
from nltk import tokenize
import tempfile
import openai
import csv

In [ ]:
nltk.download ('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import boto3
import pandas as pd

def download_and_process_data(s3_file_path):
    #remember to use your aws access and secrete keys to access Amazon's s3 bucket

    # Download the file from S3
    with open('text_data1.xlsx', 'wb') as f:
        s3.download_fileobj('tiiqu-openresearch', s3_file_path, f)

    # Read in the excel file
    text_data1 = pd.read_excel('text_data1.xlsx')

    # Drop rows with NaN values
    text_data1 = text_data1.dropna().reset_index(drop=True)

    temp_df = text_data1.copy()

    return temp_df

# Example usage of the function
#s3_file_path = 'data-ingression/Climate science_f1000_20230717.xlsx'
#text_data1 = download_and_process_data(s3_file_path)

# Now text_data1 contains the processed DataFrame


In [ ]:
# Function to tokenize the text and return the paragraphs
def get_paragraphs(text):

  # Tokenize the text
  result = tokenize.sent_tokenize(text)

  # Variables to store the tokenized strings and
  str_paragraph = ''
  paragraphs = []

  for i in range(len(result)):

    sentence = result[i]
    len_para = len(tokenize.word_tokenize(str_paragraph))

    if len_para < 200:
      str_paragraph = str_paragraph + ' ' + sentence

    elif len_para >= 200 :
      paragraphs.append(str_paragraph)

      str_paragraph = ''
      str_paragraph = str_paragraph + ' ' + sentence

    elif i == len(result) - 1:
      paragraphs.append(str_paragraph)

  # Return the paragraphs
  return paragraphs

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

# Load the OpenAI API Key
# Create a .env file storing your key
load_dotenv('openai_key.env')

False

In [ ]:
# Define the api key
openai.api_key = ('OPEN_API_KEY')

# Function to get the response from OpenAI's api
def get_response(text):
    try:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt="Generate 2 very descriptive questions and answers per paragraph wise: " + text,
            temperature=0.9,
            max_tokens=200,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].text
    except Exception as e:
        print("\nError:", str(e))
        return None

In [ ]:
# Function to get the Q&A pairs from the qna generated
def parse_qa_pairs(lines):
    pairs = []
    current_q = ''
    current_a = ''
    for line in lines:
        line = line.strip()
        if line.startswith('Q1') or line.startswith('Q2') or line.startswith('Question 1') or line.startswith('Question 2'):

            if current_q:
                pairs.append((current_q, current_a))
            current_q = line
            current_a = ''
        elif line.startswith('A1') or line.startswith('A2') or line.startswith('Answer'):

            current_a += line[2:] + '\n'

    if current_q:
        pairs.append((current_q, current_a))
    return pairs

In [ ]:
import time
import tempfile

def generate_qa_pairs(temp_df):
        # Function to get the response from OpenAI's api
        # Implementation of the get_response function here...
        # Function to get the response from OpenAI's api
  def get_response(text):
      try:
          response = openai.Completion.create(
              model="text-davinci-003",
              prompt="Generate 2 very descriptive questions and answers per paragraph wise: " + text,
              temperature=0.9,
              max_tokens=200,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
          )
          return response.choices[0].text
      except Exception as e:
          print("\nError:", str(e))
          return None

        # Function to parse Q&A pairs from lines
        # Implementation of the parse_qa_pairs function here...
  def parse_qa_pairs(lines):

      pairs = []
      current_q = ''
      current_a = ''
      for line in lines:
          line = line.strip()
          if line.startswith('Q1') or line.startswith('Q2') or line.startswith('Question 1') or line.startswith('Question 2'):

              if current_q:
                  pairs.append((current_q, current_a))
              current_q = line
              current_a = ''
          elif line.startswith('A1') or line.startswith('A2') or line.startswith('Answer'):

              current_a += line[2:] + '\n'

      if current_q:
          pairs.append((current_q, current_a))
      return pairs


    # Create a temporary DataFrame to store the cleaned data
  columns = ['Title', 'DOI', 'Authors', 'Full_Text_URL', 'Paragraph', 'Question', 'Answer']  ## Adding 'paragraph' column
  cleaned_data = pd.DataFrame(columns=columns)
  total_qapairs = 0  # Variable to store the total count of Q&A pairs

  for id, row in temp_df.iterrows():
        paragraphs = get_paragraphs(row['Full_Text_Content'])
        qapair_count = 0  # Variable to store the count of Q&A pairs per document
        new_rows = []  # List to accumulate Q&A pairs for each document

        for para in paragraphs:
            qna = []
            response = get_response(para)
            if response is not None:
                qna.append(response)
            else:
                print('\nstopping Q&A generation. . .\n')
                break

            # Create a temporary file
            with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
                for qa in qna:
                    print(qa)
                    temp_file.write(qa)
                temp_file.flush()
                temp_file_path = temp_file.name

            with open(temp_file_path, 'r') as temp_file:
                lines = temp_file.readlines()
                pairs = parse_qa_pairs(lines)
                qapair_count += len(pairs)  # Count of Q&A pairs per document

            # Accumulate the Q&A pairs for each paragraph within the inner loop
            for Q, A in pairs:
                if Q.startswith('Q1') or Q.startswith('Q2') or Q.startswith('Question 1') or Q.startswith('Question 2'):
                    new_rows.append({'Title': row['Title'], 'DOI': row['DOI'], 'Authors': row['Authors'],
                                     'Full_Text_URL': row['Full_Text_URL'], 'Paragraph': para, 'Question': Q,
                                     'Answer': A.strip()})
                elif Q.startswith('A1') or Q.startswith('A2') or Q.startswith('Answer'):
                    new_rows.append({'Title': row['Title'], 'DOI': row['DOI'], 'Authors': row['Authors'],
                                     'Full_Text_URL': row['Full_Text_URL'], 'Paragraph': para, 'Question': A.strip(),
                                     'Answer': Q})

            # Check if the limit of 100 Q&A pairs is reached and break the loop if so
            if qapair_count >= 100:
                break

            # Delay of 30 seconds between generating each Q&A pair
           # time.sleep(30)

        total_qapairs += qapair_count  # Accumulate the count of Q&A pairs for all documents

        # Concatenate all accumulated Q&A pairs to the cleaned_data DataFrame
        cleaned_data = pd.concat([cleaned_data, pd.DataFrame(new_rows)], ignore_index=True)

        print(f"Document: {row['Title']}")
        print(f"Q&A Pairs Count: {qapair_count}")
        print()

  print(f"Total Q&A Pairs Count: {total_qapairs}")
  return cleaned_data, total_qapairs

# Example usage of the function
#cleaned_data, total_qapairs = generate_qa_pairs(temp_df)


In [ ]:
# Function to store DataFrame in Google Drive
def store_dataframe_in_drive(df, file_name):
    # Define the destination file path in your Google Drive
    destination_file_path = f'/content/drive/MyDrive/{file_name}.csv'

    # Save the DataFrame to a CSV file
    df.to_csv(destination_file_path, index=False)

In [ ]:
import boto3
import pandas as pd
import time
import tempfile
from google.colab import drive



# Function to execute the whole process for multiple file paths
def process_multiple_files(file_paths):
    all_cleaned_data = []
    all_total_qapairs = []

    for file_path in file_paths:
        # Step 1: Download and process the data
        temp_df = download_and_process_data(file_path)

        # Step 2: Generate Q&A pairs for the processed data
        cleaned_data, total_qapairs = generate_qa_pairs(temp_df)

        # Store the results for this file path
        all_cleaned_data.append(cleaned_data)
        all_total_qapairs.append(total_qapairs)

    return all_cleaned_data, all_total_qapairs


# Example usage of the process_multiple_files function
if __name__ == "__main__":
    # List of file paths to be processed
    file_paths_list = ['data-ingression/Biomass_gatesopen_20230717.xlsx', 'data-ingression/Climate science_wellcome_20230717.xlsx', 'data-ingression/Green+energy_f1000_20230717.xlsx', 'data-ingression/Paris+Agreement_f1000_20230717.xlsx']

    # Execute the process for all file paths
    all_cleaned_data, all_total_qapairs = process_multiple_files(file_paths_list)

    file_name_dataframe_dict = {
        os.path.splitext(os.path.basename(file_path))[0]: df for file_path, df in zip(file_paths_list, all_cleaned_data)
    }



     # Apply string replacement operations to all dataframes in the dictionary
    for file_name, df in file_name_dataframe_dict.items():
        df['Question'] = df['Question'].str.replace(r'^[Question\d\.\:\- ]*', '', regex=True)
        df['Answer'] = df['Answer'].str.replace(r'^[Answer\d\.\:\- ]*', '', regex=True)

        # Rename the DataFrame
        renamed_df = df.rename_axis(file_name)
        from google.colab import drive

        # Mount Google Drive
        drive.mount('/content/drive')


        store_dataframe_in_drive(df, file_name)

        # Print the DataFrame name after storing in Google Drive
        print(f"DataFrame '{file_name}' has been stored in Google Drive.")





Streaming output truncated to the last 5000 lines.
A2: The steering committee is composed of eight investigators from icddr,b, three investigators from Stanford University, one investigator from Dhaka Shishu Hospital, and one investigator from Bangabandhu Sheikh Mujib Medical University.


Q1: What are the criteria for inclusion in the scientific field study?
A1: Study participants must be pregnant women 18 years of age or older in their second or third trimester of pregnancy and must have given written informed consent.

Q2: What will be done to ensure that all eligible pregnant women are included in the study?
A2: The scientific field team will go house-to-house to list all the pregnant women on the basis of documentation of their pregnancy (i.e. Antenatal care (ANC) card and/or prescription from the selected unions of a sub district) and will reach out to local government workers to identify pregnant women in 2nd or 3rd trimester.


Question 1: What is the criteria for forming clust